# 발표 소단원 리스트

8-14,8-18,8-22

# 전체 요약자료

- 클래스 기능 확장하는 방법 : 상속
- recursion 없이 작업

# 문항별 자료 
- (8-14) 사용자 정의형 container 만들기 : (8-12) 참고하면 더 좋음
- (8-18) base class에 추가적인 클래스 써서 여러 매소드들 상속하기(== mixins)
- (8-22) recursion 없이 작업하기

### 8-14) Implementing Custom Containers
- (sol) 패키지의 기본 컨테이너들 상속받기!
1. collections 라이브러리 사용 : 원하는 방식으로 매서드 정의해줘야 함!!

In [2]:
### 코드 작성
import collections
class A(collections.Iterable):
    pass

In [3]:
a = A()

TypeError: Can't instantiate abstract class A with abstract methods __iter__

In [15]:
class A(collections.Iterable):
    def __iter__(self):
        pass

In [16]:
a = A()

In [17]:
collections.Sequence()

c:\users\jihan\baselab\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


TypeError: Can't instantiate abstract class Sequence with abstract methods __getitem__, __len__

##### 아이템이 정렬된 상태로 저장되는 시퀀스 생성하는 예제

In [23]:
import collections,bisect

class SortedItems(collections.Sequence):
    def __init__(self,initial=None):
        self._items = sorted(initial) if initial is not None else []
        
    # required methods
    def __getitem__(self,index):
        return self._items[index]
    
    def __len__(self):
        return len(self._items)
    
    def add(self,item):
        bisect.insort(self._items,item)

In [24]:
items = SortedItems([5,1,3])
list(items)

[1, 3, 5]

In [25]:
items[0]

1

In [26]:
items[-1]

5

In [27]:
items.add(-10)

In [29]:
items.add(2)

In [30]:
list(items)

[-10, 1, 2, 3, 5]

In [31]:
## support iterable
for n in items:
    print(n)

-10
1
2
3
5


In [ ]:
### 장점들
# 1. type checking 하기 용이해짐
# 2. 여러 default 방법들을 기본적으로 제공!

In [33]:
class Items(collections.MutableSequence):
    def __init__(self,initial = None):
        self._items = list(initial) if initial is not None else []
        
    def __getitem__(self,index):
        print('Getting:',index)
        return self._items[index]
    
    def __setitem__(self,index,value):
        print('Setting:',index,value)
        self._items[index] = value
        
    def __delitem__(self,index):
        print('Deleting:',index)
        del self._items[index]
    
    def insert(self,index,value):
        print('Inserting:',index,value)
        self._items.insert(index,value)
        
    def __len__(self):
        print('Len')
        return len(self._items)

In [34]:
a = Items([1,2,3])
len(a)

Len


3

In [35]:
a.append(4)

Len
Inserting: 3 4


In [36]:
a.append(2)

Len
Inserting: 4 2


In [37]:
a.count(2)

Getting: 0
Getting: 1
Getting: 2
Getting: 3
Getting: 4
Getting: 5


2

In [38]:
a.remove(3)

Getting: 0
Getting: 1
Getting: 2
Deleting: 2


### (8-18) Extending Classes with Mixins
- 그 자체로 의미없는 클래스들을 추가해주기 ( init 가 없는 클래스들이여함!!)

In [48]:
class LoggedMappingMixin:
    '''
    add logging to get/set/delete operations for debugging
    '''
    
    __slots__ = ()
    
    def __getitem__(self,key):
        print('Getting ' + str(key))
        return super().__getitem__(key) ## super는 상위 상속자
    
    def __setitem__(self,key,value):
        print('Setting {} = {!r}'.format(key,value))
        return super().__setitem__(key,value)
    
    def __delitem__(self,key):
        print('Deleting ' + str(key))
        return super().__delitem__(key)
    
    
class SetOnceMappingMixin:
    '''
    Only allow a key to be set once.
    '''
    __slots__ = ()
    
    def __setitem__(self,key,value):
        if key in self:
            raise TypeError(str(key) + ' already set')
        return super().__setitem__(key,value)
    
class StringKeysMappingMixin:
    '''
    Restrict keys to strings only
    '''
    __slots__ = ()
    def __setitem__(self,key,value):
        if not isinstance(key,str):
            raise TypeError('keys must be strings')
        return super().__setitem__(key,value)

In [41]:
class LoggedDict(LoggedMappingMixin,dict):
    pass

d = LoggedDict()
d['x'] = 23

Setting x = 23


In [42]:
d['x']

Getting x


23

In [49]:
from collections import defaultdict
class SetOnceDefaultDict(SetOnceMappingMixin,defaultdict):
    pass

d = SetOnceDefaultDict(list) ## list sent to defaultdict 
d['x'].append(2)
d['y'].append(3)
d['x'].append(10)


In [50]:
d['x'] = 23

TypeError: x already set

In [55]:
from collections import OrderedDict
class StringOrderedDict(StringKeysMappingMixin,
                        SetOnceMappingMixin,
                        OrderedDict):
    pass

In [56]:
d = StringOrderedDict()
d['x']=23
d[42]=10

TypeError: keys must be strings

In [57]:
d['x']=42

TypeError: x already set

### 주의 사항
1. mixin은 그 자체로 인스턴스화 될 수 없음
2. mixin에는 init 정의 안 하는 것이 좋음
3. mixin에는 super()를 꼭 써야함!!

In [ ]:
class RestrictKeysMixin:
    def __init__(self, *args, _restrict_key_type, **kwargs):
        self.__restrict_key_type = _restrict_key_type
        super().__init__(*args, **kwargs)
    
    def __setitem__(self, key, value):
        if not isinstance(key, self.__restrict_key_type):
            raise TypeError('Keys must be ' + str(self.__restrict_key_type))
        super().__setitem__(key, value)

# Example

class RDict(RestrictKeysMixin, dict):
    pass
 
d = RDict(_restrict_key_type=str)
e = RDict([('name','Dave'), ('n',37)], _restrict_key_type=str)
f = RDict(name='Dave', n=37, _restrict_key_type=str)
print(f)
try:
    f[42] = 10
except TypeError as e:
print(e)

### class decorator 쓰는 예제

In [ ]:
def LoggedMapping(cls):
    cls_getitem = cls.__getitem__
    cls_setitem = cls.__setitem__
    cls_delitem = cls.__delitem__
    
    def __getitem__(self,key):
        print('Getting ' + str(key))
        return cls_getitem(self,key)
    
    def __setitem__(self,key,value):
        print('Setting {} = {!r}'.format(key,value))
        return cls_setitem(self,key,value)
    
    def __delitem__(self,key):
        print('Deleting ' + str(key))
        return cls_delitem(self,key)
    
    cls.__getitem__ = __getitem__
    cls.__setitem__ = __setitem__
    cls.__delitem__ = __delitem__
    return cls

## class decorator에 대해서는 (9-12)에서 자세히 다룸
@LoggedMapping
class LoggedDict(dict):
    pass

### (8-22) Implementing the Visitor Pattern Without Recursion
- (sol) stack 과 generator를 사용하기!

In [65]:
import types
class Node:
    pass

## node들 visit하기
import types
class NodeVisitor:
    def visit(self, node):
        stack = [ node ]
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                    last_result = None
                elif isinstance(last, Node):
                    stack.append(self._visit(stack.pop()))
                else:
                    last_result = stack.pop()
            except StopIteration:
                stack.pop()
        return last_result

    def _visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))

    © 2019 GitHub, Inc.
    Terms
    Privacy
    Security
    Status
    Help



SyntaxError: invalid character in identifier (<ipython-input-65-1020e760c59e>, line 35)

In [62]:
class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

# A sample visitor class that evaluates expressions
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)

    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)

    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)

    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)

    def visit_Negate(self, node):
        return -self.visit(node.operand)
    
if __name__ == '__main__':
    # 1 + 2*(3-4) / 5
    t1 = Sub(Number(3), Number(4))
    t2 = Mul(Number(2), t1)
    t3 = Div(t2, Number(5))
    t4 = Add(Number(1), t3)

    # Evaluate it
    e = Evaluator()
    print(e.visit(t4))     # Outputs 0.6

    # Blow it up

    a = Number(0)
    for n in range(1, 100000):
        a = Add(a, Number(n))

    try:
        print(e.visit(a))
    except RuntimeError as e:
        print(e)

0.6
maximum recursion depth exceeded


In [66]:
class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        yield (yield node.left) + (yield node.right)

    def visit_Sub(self, node):
        yield (yield node.left) - (yield node.right)

    def visit_Mul(self, node):
        yield (yield node.left) * (yield node.right)

    def visit_Div(self, node):
        yield (yield node.left) / (yield node.right)

    def visit_Negate(self, node):
        yield -(yield node.operand)
    
if __name__ == '__main__':
    # 1 + 2*(3-4) / 5
    t1 = Sub(Number(3), Number(4))
    t2 = Mul(Number(2), t1)
    t3 = Div(t2, Number(5))
    t4 = Add(Number(1), t3)

    # Evaluate it
    e = Evaluator()
    print(e.visit(t4))     # Outputs 0.6

    # Blow it up

    a = Number(0)
    for n in range(1, 100000):
        a = Add(a, Number(n))

    try:
        print(e.visit(a))
    except RuntimeError as e:
        print(e)

TypeError: can't send non-None value to a just-started generator

In [68]:
import types

class Node:
    pass

class Visit:
    def __init__(self, node):
        self.node = node

class NodeVisitor:
    def visit(self, node):
        stack = [ Visit(node) ]
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                    last_result = None
                elif isinstance(last, Visit):
                    stack.append(self._visit(stack.pop().node))
                else:
                    last_result = stack.pop()
            except StopIteration:
                stack.pop()
        return last_result

    def _visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))

In [69]:
class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        yield (yield Visit(node.left)) + (yield Visit(node.right))

    def visit_Sub(self, node):
        yield (yield Visit(node.left)) - (yield Visit(node.right))

    def visit_Mul(self, node):
        yield (yield Visit(node.left)) * (yield Visit(node.right))

    def visit_Div(self, node):
        yield (yield Visit(node.left)) / (yield Visit(node.right))

    def visit_Negate(self, node):
        yield -(yield Visit(node.operand))
    
if __name__ == '__main__':
    # 1 + 2*(3-4) / 5
    t1 = Sub(Number(3), Number(4))
    t2 = Mul(Number(2), t1)
    t3 = Div(t2, Number(5))
    t4 = Add(Number(1), t3)

    # Evaluate it
    e = Evaluator()
    print(e.visit(t4))     # Outputs 0.6

    # Blow it up

    a = Number(0)
    for n in range(1, 100000):
        a = Add(a, Number(n))

    try:
        print(e.visit(a))
    except RuntimeError as e:
        print(e)

0.6
4999950000


##### 보충 설명 (코멘트) 만약 있다면!!